In [1]:
library(haven)
library(tidyverse)
library(broom)
library(lme4) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.2
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand



# Missing at random

In [2]:
AA_E_Long <- read.csv('./datos/AA_E_Long.csv')
AA_NE_Long <- read.csv('./datos/AA_NE_Long.csv')

head(AA_NE_Long)

X,id,group,age,sex,weekNames,level,week,AA
<int>,<int>,<fct>,<int>,<fct>,<fct>,<int>,<int>,<fct>
1,1,teen,17,girl,ARGCalostrum,1,1,ARG
2,2,teen,16,boy,ARGCalostrum,1,1,ARG
3,3,teen,17,boy,ARGCalostrum,1,1,ARG
4,4,teen,16,boy,ARGCalostrum,1,1,ARG
5,5,teen,16,NA,ARGCalostrum,2,1,ARG
6,6,teen,16,boy,ARGCalostrum,2,1,ARG


In [3]:
missingData <- AA_NE_Long %>% mutate(`missing data` = ifelse(is.na(level), 1, 0))
head(missingData)

X,id,group,age,sex,weekNames,level,week,AA,missing data
<int>,<int>,<fct>,<int>,<fct>,<fct>,<int>,<int>,<fct>,<dbl>
1,1,teen,17,girl,ARGCalostrum,1,1,ARG,0
2,2,teen,16,boy,ARGCalostrum,1,1,ARG,0
3,3,teen,17,boy,ARGCalostrum,1,1,ARG,0
4,4,teen,16,boy,ARGCalostrum,1,1,ARG,0
5,5,teen,16,NA,ARGCalostrum,2,1,ARG,0
6,6,teen,16,boy,ARGCalostrum,2,1,ARG,0


There is not a clear distinction between patients with missing data patients and patients without:

In [4]:
numNA <- missingData %>% group_by(id, sex, age, week, AA) %>% summarise(miss = sum(`missing data`))
head(numNA)

Warning message:
“Factor `sex` contains implicit NA, consider using `forcats::fct_explicit_na`”

id,sex,age,week,AA,miss
<int>,<fct>,<int>,<int>,<fct>,<dbl>
1,girl,17,1,ALA,0
1,girl,17,1,ARG,0
1,girl,17,1,ASN,0
1,girl,17,1,ASP,0
1,girl,17,1,CYS,0
1,girl,17,1,GLN,0


In [5]:
numNA_CI <- numNA %>% filter(!is.na(sex)) %>% group_by(id, sex, age, AA) %>% summarise(total_miss = sum(miss)) %>% filter(AA == 'GLU') %>%
                    mutate(complete = ifelse(total_miss == 0, 'yes', 'no'))

numNA_CI %>% group_by(complete) %>% summarise(n())

complete,n()
<chr>,<int>
no,32
yes,29


In [6]:
numNA_CI %>% group_by(complete, sex) %>% summarise(n())

complete,sex,n()
<chr>,<fct>,<int>
no,boy,19
no,girl,13
yes,boy,16
yes,girl,13


In [21]:
weekSexNA <- numNA %>% filter(!is.na(sex)) %>% group_by(id, sex, age, week, AA) %>% summarise(total_miss = sum(miss)) %>% filter(AA == 'GLU') %>%
                    mutate(complete = ifelse(total_miss == 0, 'yes', 'no')) %>%
                    group_by(complete, week, sex) %>% summarise(total = n())

### Sex distribution for patients with missing data and without by week:

In [22]:
weekSexNA %>% spread(sex, total) %>% filter(week == 1) %>% select(-week)
weekSexNA %>% spread(sex, total) %>% filter(week == 2) %>% select(-week)
weekSexNA %>% spread(sex, total) %>% filter(week == 8) %>% select(-week)
weekSexNA %>% spread(sex, total) %>% filter(week == 16) %>% select(-week)

Adding missing grouping variables: `week`


week,complete,boy,girl
<int>,<chr>,<int>,<int>
1,yes,35,26


Adding missing grouping variables: `week`


week,complete,boy,girl
<int>,<chr>,<int>,<int>
2,no,8,6
2,yes,27,20


Adding missing grouping variables: `week`


week,complete,boy,girl
<int>,<chr>,<int>,<int>
8,no,12,11
8,yes,23,15


Adding missing grouping variables: `week`


week,complete,boy,girl
<int>,<chr>,<int>,<int>
16,no,15,9
16,yes,20,17


In [23]:
weekSexPlot <- weekSexNA %>% spread(sex, total) %>% mutate(proportion = round(boy/(boy + girl), 2)) %>%
                    select(complete, proportion, week) %>% spread(complete, proportion)

weekSexPlot

week,no,yes
<int>,<dbl>,<dbl>
1,NA,0.57
2,0.57,0.57
8,0.52,0.61
16,0.62,0.54


In [11]:
# Week 2: Identical Proportions:
14 * 0.57
14 * 0.57 * (1 - 0.57)


47 * 0.57
47 * 0.57 * (1 - 0.57)

# N(np, npq):
# Week 8: Overlapping 95% CIs under the normal approximation.
23 * 0.52
23 * 0.52 * (1 - 0.52)


38 * 0.61
38 * 0.61 * (1 - 0.61)

# Week 16: Overlapping 95% CIs under the normal approximation.
24 * 0.62
24 * 0.62 * (1 - 0.62)


37 * 0.54
37 * 0.54 * (1 - 0.54)



# Missing data increases with time

The number of observations per week is bigger than 37

In [10]:
AA_NE_Long %>% filter(!is.na(level), AA == 'GLU') %>% group_by(week) %>% summarise(n_obs = n())

week,n_obs
<int>,<int>
1,65
2,48
8,39
16,37
